In [1]:
"""" Importing Libraries """

import speech_recognition as sr
import pyaudio 
import datetime
import time
import requests
import json
import os
import random

In [2]:
"""" Dialog Flow Credentials and other setup"""

os.environ["GOOGLE_APPLICATION_CREDENTIALS"]= r"Path to Json Credential"
DIALOGFLOW_PROJECT_ID = 'Your Project Id'
DIALOGFLOW_LANGUAGE_CODE = 'en-US'
SESSION_ID = 'any user specified id' ## can be any manual id specified by user
# Loading Api.txt Key obtained from Google Cloud Platform
with open('apikey.txt') as f:
    api_key = f.readline()
    f.close
counter=0

In [3]:
"""" Speak Function For Speech to Text 
    It uses google Speech to Text APi"""

def speak():
    
    time.sleep(3)
    intro = ['Please Speak!', 'I am Listening', 'Go ahead']  ##  Random Greet Message

    print(random.choice(intro))
    rec = sr.Recognizer() # Creating Instance of recognizer 
    mic = sr.Microphone() # Creating Instace of Microphone

# Listening to User 
    with mic as inputMethod:
        rec.adjust_for_ambient_noise(inputMethod) # Setting Auto threshold to cut off silence 
        rec.pause_threshold = 0.8
        record = rec.listen(inputMethod)

# Initializing response object
    response = {"Status": True,
                "error": None,
                "transcribe": None}

# try: for recognizing the speech obtained via user
# Exception: for RequestError or UnknownValueError if exists,
# Updating Response object respectively
    try:
        response["transcribe"] = rec.recognize_google(record)
        
    except sr.RequestError:
# API was unreachable or unresponsive
        response["Status"] = False
        response["error"] = "API Connectivity Issue"
        print("ERROR: {}".format(response["error"]))
    except sr.UnknownValueError:
# speech was unintelligible
        response["error"] = "Speech is unclear! Can you repeat again"
        print("{}".format(response["error"]))
        response=speak()
        
    return(response)


In [4]:
"""" Detect intent function: It Detects the intent from the passed text
     Intents are specified in the Dialog flow agent """

def detect_intent_texts(project_id, session_id, texts, language_code):  
    import dialogflow_v2 as dialogflow
    import json
    session_client = dialogflow.SessionsClient()

    session = session_client.session_path(project_id, session_id)
    
    text_input = dialogflow.types.TextInput(
        text=texts, language_code=language_code)

    query_input = dialogflow.types.QueryInput(text=text_input)

    response = session_client.detect_intent(
        session=session, query_input=query_input)

    print('You Said: {}'.format(response.query_result.query_text))
    return (response)

In [5]:
"""Function to obtain Entities name and type using Google NLP API Entity analysis"""

def ent(text):
    import six
    from google.cloud import language
    from google.cloud.language import enums
    from google.cloud.language import types

    client = language.LanguageServiceClient()

    if isinstance(text, six.binary_type):
        text = text.decode('utf-8')

# Instantiates a plain text document.
    document = types.Document(
        content=text,
        type=enums.Document.Type.PLAIN_TEXT)

# Detects entities in the document
    entities = client.analyze_entities(document).entities
    entity_list=[]
    entity_type_name=[]
    
    if not entities:
        print("No entities found, Please try again")
    ### Put the link for furhter location and other stuff check
    
    else:
        for entity in entities:
            entity_type = enums.Entity.Type(entity.type)
            entity_list.append(entity.name)
            entity_type_name.append(entity_type.name)
            print('------' * 10)
    return(entity_list,entity_type_name)


In [6]:
""" Weather agent function: Takes the user input for variable location and displys the result  """

def weather_agent():
       
    weather_subcheck='LOCATION'
    print('\n\n')

    print('Great \n\n')

    print("You want to check weather of which city ?\n\n")
    r1=speak()
    entities_R1=ent(r1["transcribe"])
    print("You said :{}\n".format(r1["transcribe"]))
    idx=0
    idj=0
    check=[]
## Obtain the index of Entity Type 'Location' from 
    for i in entities_R1:
        idx+=1
        if (weather_subcheck in i):
            idj=i.index('LOCATION')
        for j in i:
            check.append(j)
## Check for Entity type 'LOCATION'        
    if (weather_subcheck in check):
## Passing Location info based on its index to Function report() which will fetch weather report
        if (idx==1):
            locat=entities_R1[(idx-1)]
            weather_report=report(locat) ## Calls Report ()
        elif (idx>1):
            locat=entities_R1[(idx-2)][idj]
            weather_report=report(locat)  ## Calls Report ()
## Speech Recongition did not catch location info
    else:
        print("Didnt Get the name of the city can you repeat it again\n")
        weather_agent()


In [7]:
"""Report Function: It calls Yahoo Weather API to fetch the responses based on user inputs
   which in our case is location """

def report(x):
    import time, uuid, urllib, json
    import hmac, hashlib
    from base64 import b64encode

    ## Passes yahoo credentials 
    with open('Yahoo_Credentials.json') as f: 
        data = json.load(f)


    app_id = data['app_id']
    consumer_key = data['consumer_key']
    consumer_secret = data['consumer_secret']
    concat = '&'
    query = {'location':x, 'format': 'json', 'u':'c'}
    url = data['url']
    method = 'GET'


    oauth = {
    'oauth_consumer_key': data['consumer_key'],
    'oauth_nonce': uuid.uuid4().hex,
    'oauth_signature_method': 'HMAC-SHA1',
    'oauth_timestamp': str(int(time.time())),
    'oauth_version': '1.0'
    }

    # Prepare signature string (merge all params and SORT them)
    merged_params = query.copy()
    merged_params.update(oauth)
    sorted_params = [k + '=' + urllib.parse.quote(merged_params[k], safe='') for k in sorted(merged_params.keys())]
    signature_base_str = method + concat + urllib.parse.quote(url, safe='') + concat + urllib.parse.quote(concat.join(sorted_params), safe='')

    # Generate signature
    composite_key = urllib.parse.quote(consumer_secret, safe='') + concat
    oauth_signature = b64encode(hmac.new(composite_key.encode('utf-8'), signature_base_str.encode('utf-8'), hashlib.sha1).digest())

    # Prepare Authorization header
    oauth['oauth_signature'] = oauth_signature.decode('utf-8')
    auth_header = 'OAuth ' + ', '.join(['{}="{}"'.format(k,v) for k,v in oauth.items()])

    # Send request
    url = url + '?' + urllib.parse.urlencode(query)

    request = urllib.request.Request(url)
    request.headers['Authorization'] = auth_header
    request.headers['X-Yahoo-App-Id']= app_id

    
    """ Responses from Weather API are displayed """
    
    
    response = urllib.request.urlopen(request).read()
    response_list=json.loads(response)

    print("\n\n\n\n")
    print("For Today:\n")
    print("It will be {}\n".format(response_list['current_observation']['condition']['text']))
    print("The temperature will be {}\n\n\n".format(response_list['current_observation']['condition']['temperature']))

    print("Forecast for next 5 days\n\n\n")
    for elements in response_list['forecasts'][0:5]:
        day="Monday" if elements['day']=='Mon' else "Tuesday" if elements['day']=='Tue' else "Wednesday" if elements['day']=='Wed' else "Thursday" if elements['day']=='Thu' else "Friday" if elements['day']=='Fri' else "Saturday" if elements['day']=='Sat' else "Sunday"
        print("For {}:\n".format(day))
        print("It will be {}\n".format(elements['text']))
        print("The Low will be {l} and high will be {h}\n\n".format(l=elements['low'],h=elements['high']))

    


In [8]:



""" Continue Exit funciton: It checks whether User wants to continue the converstaion or not
    and Terminates the Chat bot Function based on user input 
"""

def continue_Exit():
    print("\n\n\n\n\nDo you want to look for any more things? \n\nSay 'Yes' To contiue 'No' To exit")
    print("\n\n\n\n\n")
    continue_bot=speak()
    continue_bot=continue_bot["transcribe"]
    if (continue_bot=='yes'):
        print("\n\nOkay! So what you want to look for")
        chatBot()
    elif (continue_bot=='no'):
        print("\n\nHave a Great Day! Bah Bye")
    else:
        print("Didnt Get your response? can you repeat that")
        continue_Exit()
    



In [9]:
""" Greeting as per time """ 

hour = datetime.datetime.now().hour
if 5<=hour< 12:
    greet = "Good Morning"
elif hour < 18:
    greet = "Good Afternoon"
else:
    greet = "Good Evening"
    

In [10]:
"""  Intents Associated with Scenarios  """

"""Restaurant Search intents"""

venues_list=['venues.eating_out.search', 
                     'venues.nightlife.search',
                     'venues.search - location',
                     'venues.search - open',
                     'venues.search - sort']
            
             
"""   Weather Report Search intents  """

weather_list=['weather',
     'weather - context:weather - comment:activity',
     'weather - context:weather - comment:address',
     'weather - context:weather - comment:address & date-time',
     'weather - context:weather - comment:condition',
     'weather - context:weather - comment:date time',
     'weather - context:weather - comment:outfit',  
     'weather.activity',
     'weather.condition',
     'weather.outfit',
     'weather.temperature']

"""   News Report Search intents  """
             
news_list = ['news.search',  
            'news.search - context:news-search']


"""  Translate Report Intents  """
             
             
translate_list=['translate.text',  
 'translate.text - context:translate-text - comment:from language',
 'translate.text - context:translate-text - comment:text',
 'translate.text - context:translate-text - comment:to language']

"""    Jokes intent  """
jokes_list=['jokes.get']




In [11]:
"""  Type of restautant Function:: Take User input for type of cuisine and location  and
     calls other function like restaurant_list (), Direction_restaurant() """

def type_of_restaurant():    
                        
            print('Great \n\n')
            print("Which type of cuisine are you looking for ? Like Mexican or Indian \n\n")
            r1=speak()
            res1=r1["transcribe"]
            parameters1=get_parameters(res1)
             ##Check fot dictonary key is empty or not
            if (bool(parameters1.get('cuisine'))):
                cuisine=(parameters1['cuisine'][0])
                print("You want list of nearby Restaurants or any specific location? \n\n")
                r2=speak()
                res2=r2["transcribe"]
                parameters2=get_parameters(res2)
                check1=['nearest']
                check2='restaurant'
                if (bool((parameters2['sort'])==check1)):
                    print("Awesome, Let me fetch list of nearby restaurants for you \n\n")
                    current_coordinates=getLocation()
                    restaurant_list(current_coordinates[0],cuisine) ## Calls restaurant_list()
                    direction_to_destination()                  ## Calls direction_to_destination()

                elif (((bool(parameters2.get('city'))) or(bool(parameters2.get('location')))) and (bool((parameters2['venue-type'])==check2))):

                    if (bool(parameters2.get('city'))):
                        new_pos=parameters2['city']

                    else:
                        new_pos=parameters2.get('location')['subadmin-area']
                        new_coordinate=geo_code(new_pos)
                        restaurant_list(new_coordinate[0],cuisine) ## Calls restaurant_list()
                        direction_to_destination()         ## Calss direction_to_destination()
                else :
                    print("sorry , Can not get what your saying , Speak again")
                    type_of_restaurant()   ## Calls it self
                               
            else :
                print("sorry, Can you repeat the cusinie type again")
                type_of_restaurant() ## Calls it self
    

In [12]:
""" Geo_code functions: It displays the Coordinates based on the Location name 
    it uses Google Geocode API """


def geo_code(loc):
    lk='https://maps.googleapis.com/maps/api/geocode/json?address='+loc+'&key='+api_key
    r = requests.get(lk)
    t=json.loads(r.text)
    ppp=[]
    lat=t['results'][0]['geometry']['location']['lat']
    lng=t['results'][0]['geometry']['location']['lng']
    pos=str(lat)+","+str(lng) ## String Format
    pos1=lat,lng              ## Tuble Format
    ppp.append(pos)
    ppp.append(pos1)
    return(ppp)

In [13]:

""" Direction to Destination Function: It takes Users input for Restaurant Selected and 
    Gives the direction to the desired restaurant
    Display Direction and Shows the route in Google Map"""

def direction_to_destination():

    import googlemaps
    from bs4 import BeautifulSoup
    from datetime import datetime
    gmaps = googlemaps.Client(key=api_key)
    print("\n\nPlease select the restaurant that you want to go to\n\n")
    
    ## Selected User input for Restaurants 
    r4=speak()
    restaurant_dest=r4['transcribe']
    current_loc=getLocation()            ## Call getLocation() to get current location coordinates
    destination=geo_code(restaurant_dest)   ## Call geo_code() to get the destination codes coordinates
    print("Your Selected Restaurant is: {}".format(restaurant_dest))
    # Request directions via public transit
    now = datetime.now()
    
    ## Mode Seleceted is Driving
    directions_result = gmaps.directions(current_loc[1],
                                         destination[1],
                                         mode="driving",
                                         departure_time=now)
    print("Total distance is {}".format(directions_result[0]['legs'][0]['distance']['text']))
    print("Time to reach destination: {}".format(directions_result[0]['legs'][0]['duration']['text']))
    print("Travel Mode : Driving")
    stepss=directions_result[0]['legs'][0]['steps']
    for x in range(len(stepss)):
        instruction=stepss[x]['html_instructions']
        soup = BeautifulSoup(instruction)
        print ("\n {}".format(soup.get_text()))
        print("\n After:{} ".format(stepss[x]['distance']['text']))
        if x==(len(stepss)-1):
            print("\nYou have reached your Destination")
        else:
            False
            
    ## Display the Map with a layer which shows Current location and Destination        
    direction_plot(current_loc[1],destination[1])
    


In [14]:
"""  GetLocation Function: It will provide the current Location Coordinates 
     It uses Google Geolocation ApI """


def getLocation(): 
    get_origin = requests.post(
                "https://www.googleapis.com/geolocation/v1/geolocate?key={}".format(api_key)).json()
    origin_lat = get_origin['location']['lat']
    origin_lng = get_origin['location']['lng']
    location = str(origin_lat) + ',' + str(origin_lng) ## String Format
    location1=origin_lat,origin_lng                    ## Tuple Format
    lll=[]
    lll.append(location)
    lll.append(location1)
    return (lll)

In [15]:
"""   Restaurant_list Function: It will print List of restaurant based on User inputs
     User Inputs are:
     location : Like nearby or Specific location 
     Cuisine: Type of Cuisine (Like Mexican or Indian )
    
    We are using Google Place Api 
"""

def restaurant_list(pos,cuisine):
    link='https://maps.googleapis.com/maps/api/place/nearbysearch/json?location='
    complete_link=link+pos+'&radius=3500&type=restaurant&keyword='+cuisine+'&key='+api_key
    r = requests.get(complete_link)
    x = r.json() 

    # now x contains list of nested dictionaries 
    # we know dictionary contain key value pair 
    # store the value of result key in variable y 
    y = x['results'] 

    # keep looping upto lenght of y 
    for i in range(len(y)): 

        # Print value corresponding to the 
        # 'name' key at the ith index of y 
        print("\n")
        print(y[i]['name']) 


In [16]:
"""  Joke_display Function: It will print Joke on screen"""


def jokes_display():

    from bs4 import BeautifulSoup
    with open('jokes.txt') as f:
        jokes = f.read()
        f.close
    soup = BeautifulSoup(jokes)
    t=soup.get_text()
    l=t.split("\n\n")
    print("\n")
    print(random.choice(l))


In [17]:
"""  Text Translate Function : It Translates Text from English to target Language
    It uses Google Translate to Text API """

def text_translate():
# Imports the Google Cloud client library
    from google.cloud import translate
    # Instantiates a client
    translate_client = translate.Client()
    print("\nYou can select the language you want to Translate to from these :")
    print("\n Languages to select: Hindi, Chinese, French, German")
     
    language_select=speak()
    language_select=language_select["transcribe"]
    languages = {"Hindi" : 'hi',
                "Chinese" : 'zh-CN',
                "French" : 'fr',
                "German" : 'de'}
    if (language_select in languages.keys()): 
        lang_code=languages[language_select]
        time.sleep(5)
        print("\nNote: English is the default Input Language")
        print("\nPlease say the text that you want to Translate")
        text_To_Translate=speak()
        text_To_Translate=text_To_Translate["transcribe"]
         # Translates some text into language
        translation = translate_client.translate(text_To_Translate,target_language=lang_code)

        print(u'Text: {}'.format(text_To_Translate))
        print(u'Translation: {}'.format(translation['translatedText']))
        
    else:
        print("\nSelected language is not defined or Speech is unclear")
        print("\nTry again")
        text_translate()
        

In [18]:
""" Newsreport Function: It calls News api and display responses """

def news_report():
    with open('news_api.txt') as f:   ## Pass New_api Credentials 
        news_api_key = f.readline()
        f.close
    url = ('https://newsapi.org/v2/top-headlines?'
           'country=us&'
           'apiKey='+news_api_key)
    response = requests.get(url)
    response=response.json()
    news_fetch=response['articles']
    for i in range(len(news_fetch)):
        print("\n\n{} News Source : {}".format(i+1,(news_fetch[i]['source']['name'])))
        print("\n\nTitle description: {}".format((news_fetch[i]['description'])))

In [19]:
""" get_parameters function : Gets the parameters from Dialog flow agent based on the input text 
"""

def get_parameters(responses):
    ###Protocol Buffers is a method of serializing structured data. 
    ### It is useful in developing programs to communicate
    ### with each other over a wire or for storing data
    
    import google.protobuf  as pf
    
    para=detect_intent_texts(DIALOGFLOW_PROJECT_ID,SESSION_ID,responses,DIALOGFLOW_LANGUAGE_CODE)
    para=pf.json_format.MessageToJson(para.query_result.parameters, including_default_value_fields=False)
    para=json.loads(para)
    return(para)

In [20]:
""" Chatbot function: It Incorporates all function we have defined """

def chatBot():
    global counter
    if (counter==0):
        Greet_msg = (
            "A Very {g} to you! \n"
            "My Name is Harry \n"
            "How can I help you today\n"
            "Say 'Your command' for furhter interaction".format(g=greet))

        print(Greet_msg)
    else:
        pass
    
    response=speak()
    responses=response["transcribe"]
    counter+=1

    print("\n\n")
    test1=detect_intent_texts(DIALOGFLOW_PROJECT_ID,SESSION_ID,responses,DIALOGFLOW_LANGUAGE_CODE)
    intent_check=test1.query_result.intent.display_name
    if ( intent_check in weather_list):
        weather_agent()
        time.sleep(10)
        continue_Exit()
    elif (intent_check in venues_list):
        type_of_restaurant()
        time.sleep(20)
        continue_Exit()
    elif( intent_check in news_list):
        print("Fetching News Please wait\n\n")
        news_report()
        time.sleep(10)
        continue_Exit()
    elif( intent_check in translate_list):
        print("The Input language is English\n\n")
        text_translate()
        time.sleep(10)
        continue_Exit()
    elif( intent_check in jokes_list):
        print("All Set ... Here it comes\n\n")
        jokes_display()
        time.sleep(10)
        continue_Exit()
    else:
        print("\n\n Did not get you, Please speak again\n\n") 
        chatBot()
    

In [25]:
from IPython.display import display
import gmaps
import gmaps.datasets
fig = gmaps.Map()

""" Direction plot Function: It displays Current location and Destination location in MAP
    it takes in Current location coordiantes and Destination cordinates """

def direction_plot(current_loc,destination):
    global api_key
    gmaps.configure(api_key) 
#     gmaps.configure(api_key=os.environ["GOOGLE_APPLICATION_CREDENTIALS"])
    direction = gmaps.directions_layer(current_loc, destination)
    fig.add_layer(direction)
    fig
    display(fig)
    

In [ ]:
counter=0
chatBot()